In [133]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
import sys

#Set device
USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    device = torch.device("cuda")
    cuda = True
else:
    device = torch.device("cpu")
    cuda = False
    
print("Device =", device)
gpus = [0]

Device = cuda


In [2]:
def time_elapsed(start_time):
    elapsed = time.time() - start_time
    hours = int(elapsed/3600)
    minutes = int(int(elapsed/60)%60)
    seconds = int(elapsed%60)
    
    return hours, minutes, seconds

In [3]:
#Load SMILES data as integer labels and as one-hot encoding
data = np.load("ohesmiles.npz")
data = data["arr_0"]

intdata = np.load("intsmiles.npz")
intdata = intdata["arr_0"]

data = torch.from_numpy(data).view(np.shape(data)[0], 1, np.shape(data)[1])
intdata = torch.from_numpy(intdata)

print("Dataset size: " + str(data.size()))
print("Integer dataset size: " + str(intdata.size()))

Dataset size: torch.Size([34131372, 1, 55])
Integer dataset size: torch.Size([34131372])


In [109]:
#Get input tensor
def inptarg(i, shuffle, batch_size):
    
    #Define input and target tensor sizes
    inp = torch.Tensor((seq_length - 1) * batch_size, 1, np.shape(data)[2])
    target = torch.Tensor((seq_length - 1) * batch_size)
    
    #SMILES molecules in batch 
    inputs = data[int(shuffle[i] * seq_length) : int((shuffle[i] * seq_length) + (seq_length * batch_size)), :, :]
    targets = intdata[int(shuffle[i] * seq_length) : int((shuffle[i] * seq_length) + (seq_length * batch_size))]
    
    #Index counters for input, target
    r = 0
    s = 0 
    for p in range(seq_length * batch_size - 1):
        
        if (p % 76 != 75):
            #Input data (does not include last character in SMILES)
            inp[r, :, :] = inputs[p, :, :]
            r += 1
            
        if (p % 76 != 0):
            #Target data (does not include first character in SMILES)
            target[s] = targets[p]
            s += 1

    return inp, target

In [5]:
#Get target tensor
def target(i, shuffle):
    
    #Target (does not include first character in SMILES)
    target = intdata[int((shuffle[i] * seq_length) + 1) : int((shuffle[i] * seq_length) + seq_length)]

    return target

In [127]:
#Define model
class Model(nn.Module):
    
    #Define model parameters
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Model, self).__init__()
        
        #Model parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        
        #Model layers
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        self.linear = nn.Linear(hidden_size, input_size)
        
    #Define initial hidden and cell states
    def init_states(self, num_layers, hidden_size):
        hidden = [Variable(torch.zeros(num_layers, 1, hidden_size)),
                  Variable(torch.zeros(num_layers, 1, hidden_size))]
        
        return hidden
    
    #Define forward propagation
    def forward(self, inp, hidden):
        output, hidden = self.lstm(inp, hidden)
        output = self.linear(output)
        
        return output, hidden
    

In [135]:
#Set start time
start_time = time.time()

#Define training
def train(epochs):
    #Iterate over desired number of epochs 
    for e in range(epochs):
        
        #Get random order of SMILES molecules (shuffle data)
        shuffle = np.arange(np.shape(data)[0] / seq_length)
        random.shuffle(shuffle)
        
        #Iterate over each molecule in dataset
        for i in range(int(np.shape(data)[0] / seq_length / batch_size)):
            
            #Initialize hidden and cell states
            hidden = model.init_states(num_layers, hidden_size)
            
            #Run on GPU if available
            if cuda:
                hidden = (hidden[0].cuda(), hidden[1].cuda())
        
            #Set initial gradients
            model.zero_grad()
    
            #Set initial loss
            loss = 0 
            
            #Get input and target
            input_data, target_data = inptarg(i, shuffle, batch_size)
            input_data = input_data.float()
            target_data = target_data.long()
            
            #Run on GPU if available
            if cuda:
                input_data = input_data.cuda()
                target_data = target_data.cuda()
                
            #Run model, calculate loss
            output, hidden = model(input_data, hidden)
            loss += criterion(output.squeeze(), target_data.squeeze())
                
            #Backpropagate loss
            loss.backward()
            optimizer.step()
            
            #Update list of losses
            if (e == i == 0):
                losses[0] = loss.data.item() / seq_length   
            losses.append(loss.data.item() / seq_length)
            
            if (i % 10 == 0):
                torch.save(model.state_dict(), "LSTMtrained.pth")
            
            #Print training info
            hours, minutes, seconds = time_elapsed(start_time)
            print("Loss: {:0.6f}".format(loss.data.item() / seq_length) + " | ΔLossTotal: {:+0.4f}".format(losses[-1] - losses[1]) + " | Epoch: {0:02d}".format(e + 1) + " | Iteration: {0:04d}".format(i + 1) + " | Time elapsed: {0:02d}".format(hours) + "h {0:02d}".format(minutes) + " m {0:02d}".format(seconds) + " s")
        

In [136]:
#Initialize model parameters
input_size = np.shape(data)[2]
hidden_size = 256
num_layers = 3
dropout = .2
learning_rate = 0.001
epochs = 1
seq_length = 76
batch_size = 32

#List of losses
losses = [0]

In [137]:
#Call model, set optimizer and loss function
model = Model(input_size, hidden_size, num_layers, dropout)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

#Run on GPU if available
if cuda:
    model.cuda()
    criterion.cuda()  

In [138]:
#Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters in network: " + str(total_params))

Total number of parameters in network: 1387319


In [139]:
#Train
train(epochs)

Loss: 0.052533 | ΔLossTotal: +0.0000 | Epoch: 01 | Iteration: 0001 | Time elapsed: 00h 00 m 03 s
Loss: 0.052119 | ΔLossTotal: -0.0004 | Epoch: 01 | Iteration: 0002 | Time elapsed: 00h 00 m 05 s
Loss: 0.051464 | ΔLossTotal: -0.0011 | Epoch: 01 | Iteration: 0003 | Time elapsed: 00h 00 m 06 s
Loss: 0.050106 | ΔLossTotal: -0.0024 | Epoch: 01 | Iteration: 0004 | Time elapsed: 00h 00 m 07 s
Loss: 0.048085 | ΔLossTotal: -0.0044 | Epoch: 01 | Iteration: 0005 | Time elapsed: 00h 00 m 09 s
Loss: 0.043834 | ΔLossTotal: -0.0087 | Epoch: 01 | Iteration: 0006 | Time elapsed: 00h 00 m 10 s
Loss: 0.040249 | ΔLossTotal: -0.0123 | Epoch: 01 | Iteration: 0007 | Time elapsed: 00h 00 m 11 s
Loss: 0.031045 | ΔLossTotal: -0.0215 | Epoch: 01 | Iteration: 0008 | Time elapsed: 00h 00 m 12 s
Loss: 0.036060 | ΔLossTotal: -0.0165 | Epoch: 01 | Iteration: 0009 | Time elapsed: 00h 00 m 14 s
Loss: 0.031616 | ΔLossTotal: -0.0209 | Epoch: 01 | Iteration: 0010 | Time elapsed: 00h 00 m 15 s
Loss: 0.029021 | ΔLossTotal: -

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "LSTMtrained.pth")